In [34]:
from utils.custom_dataset import YOLO6ChannelDataset

train_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/train/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/train/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/train"
)

val_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/val/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/val/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/val"
)

In [35]:
import torch
from torch.utils.data import DataLoader
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 8
save_path = "yolov8_custom6.pt"


# --- Collate Function ---
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs, dim=0)
    targets_with_id = []
    for i, t in enumerate(targets):
        if t.numel() == 0:
            continue
        img_idx = torch.full((t.shape[0], 1), i, dtype=t.dtype)
        targets_with_id.append(torch.cat([img_idx, t], dim=1))
    if len(targets_with_id) == 0:
        targets_combined = torch.zeros((0, 6))
    else:
        targets_combined = torch.cat(targets_with_id, dim=0)
    # print(f"targets_combined: {targets_combined}")
    return imgs, targets_combined


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_fn, num_workers=0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_fn, num_workers=0)


# --- Model ---
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8n.yaml', ch=6, nc=6).to(device)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.to(device)
# print(f"Expected output channels: {(model.args.reg_max + 1) * 4 + model.nc}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# --- Training Loop ---
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    
    for imgs, targets in pbar:
        # print(f"imgs.shape: {imgs.shape}")
        imgs = imgs.to(device)
        targets = targets.to(device)

        if targets.shape[0] == 0:
            continue

        optimizer.zero_grad()
        preds = model(imgs)

        batch = {
            "img": imgs,
            "batch_idx": targets[:, 0],
            "cls": targets[:, 1],
            "bboxes": targets[:, 2:]
        }
        
        loss, loss_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
        loss = loss.sum()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            if targets.shape[0] == 0:
                continue

            preds = model(imgs)
            batch = {
                "img": imgs,
                "batch_idx": targets[:, 0],
                "cls": targets[:, 1],
                "bboxes": targets[:, 2:]
            }
            
            loss_out, loss_out_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
            loss_out = loss_out.sum()
            val_loss += loss_out.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"🧪 Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}")

    # --- Save ---
    if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
        model.save('yolov8_custom6.pt')
        print(f"✅ Model saved to: {save_path}")


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       896  ultralytics.nn.modules.conv.Conv             [6, 16, 3, 2]                 


  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256,

Epoch 1/50: 100%|██████████| 17/17 [06:21<00:00, 22.42s/it, loss=361]


📘 Epoch 1 - Train Loss: 482.9622
🧪 Epoch 1 - Val Loss: 423.1375


Epoch 2/50: 100%|██████████| 17/17 [06:55<00:00, 24.42s/it, loss=338]


📘 Epoch 2 - Train Loss: 448.5152
🧪 Epoch 2 - Val Loss: 422.4111


Epoch 3/50:  18%|█▊        | 3/17 [01:29<06:58, 29.93s/it, loss=437]


KeyboardInterrupt: 

In [ ]:
import torch
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace

# Configuration
weights_path = "yolov8_custom6.pt"  # Path to your trained model weights
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load your 6-channel YOLOv8 model with 6 classes
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8.yaml', ch=6, nc=6)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.load_state_dict(torch.load(weights_path, map_location=device))
model.eval()
model.to(device)

In [23]:
from torchvision import transforms as T

transform = T.Compose([
    T.Resize((640, 640)),  # Adjust if your model uses a different size
    T.ToTensor()
])

def load_6ch_image(path_left, path_right):
    img_right = transform(Image.open(path_right).convert("RGB"))  # (3, H, W)
    img_left = transform(Image.open(path_left).convert("RGB"))  # (3, H, W)
    return torch.cat([img_right, img_left], dim=0)  # (6, H, W)

In [32]:
import torch
import cv2
from ultralytics.utils.ops import non_max_suppression

def predict_and_draw(img_path_right, img_path_left, output_path, conf_thresh=0.1, iou_thresh=0.5):
    # Load your custom 6-channel image (combine right+left)
    img_6ch = load_6ch_image(img_path_right, img_path_left).unsqueeze(0).to(device)  # (1, 6, H, W)
    
    model.eval()
    with torch.no_grad():
        raw_preds = model(img_6ch)  # raw model outputs, shape: (1, 10, 8400) or similar

        # Apply postprocessing - decode + NMS in one step
        # Note: Use your model's 'postprocess' method if available, else use NMS directly on raw preds
        # Since DetectionModel has no decode/postprocess method, use non_max_suppression here:
        preds = non_max_suppression(raw_preds, conf_thres=conf_thresh, iou_thres=iou_thresh, nc=6)[0]  # nc=number classes

        print(f"Detections shape: {preds.shape if preds is not None else 'No detections'}")
        print(preds)

    # Load original right image for visualization (where you want boxes)
    original = cv2.imread(img_path_right)

    if preds is not None and preds.shape[0] > 0:
        for *xyxy, conf, cls in preds:
            # xyxy are the bounding box corners
            label = f"{int(cls.item())}: {conf.item():.2f}"
            # Draw bounding box
            cv2.rectangle(original, (int(xyxy[0]), int(xyxy[1])), (int(xyxy[2]), int(xyxy[3])), (0, 255, 0), 2)
            # Draw label text
            cv2.putText(original, label, (int(xyxy[0]), int(xyxy[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (0, 255, 0), 1, cv2.LINE_AA)
    else:
        print("No objects detected")

    # Save image with detections
    cv2.imwrite(output_path, original)


In [33]:
input_folder = "Yolo/data/test_for_prediction"
output_folder = "Yolo/runs/predict_custom_6ch_v8"
os.makedirs(output_folder, exist_ok=True)

file_names = [f for f in os.listdir(input_folder + "/right") if ".png" in f]

for file_name in file_names:
    right_path = os.path.join(input_folder + "/right", file_name)
    left_path = os.path.join(input_folder + "/left", file_name)
    output_path = os.path.join(output_folder, file_name.replace(".png", "_pred.png"))

    predict_and_draw(right_path, left_path, output_path)
    print(f"Saved: {output_path}")


Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5377_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5387_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5397_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5407_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5417_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5427_pred.png
Detections shape: torch.Size([0, 6])
tensor([], size=(0, 6))
No objects detected
Saved: Yolo/runs/predict_custom_6ch_v8\Image_5437_pred.png
Detections shape: to

KeyboardInterrupt: 